In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import warnings
warnings.filterwarnings("ignore")


In [2]:
btc_file = '../knn/Bitcoin_01.10.2017-20.03.2025_historical_data_coinmarketcap.csv'
eth_file = '../knn/Ethereum_01.12.2018-28.02.2025_historical_data_coinmarketcap.csv'

df_btc = pd.read_csv(btc_file, sep=';')
df_eth = pd.read_csv(eth_file, sep=';')

scaler = MinMaxScaler()
scaled_df = df_btc.copy()
scaled_df[['open', 'high', 'low', 'close', 'volume', 'marketCap']] = scaler.fit_transform(
    scaled_df[['open', 'high', 'low', 'close', 'volume', 'marketCap']]
)

# scaled_df['timeOpen'] = pd.to_datetime(scaled_df['timeOpen'], format='ISO8601')
# scaled_df['timeClose'] = pd.to_datetime(scaled_df['timeClose'], format='ISO8601')
# scaled_df['timeHigh'] = pd.to_datetime(scaled_df['timeHigh'], format='ISO8601')
# scaled_df['timeLow'] = pd.to_datetime(scaled_df['timeLow'], format='ISO8601')
# scaled_df['timestamp'] = pd.to_datetime(scaled_df['timestamp'], format='ISO8601')
scaled_df.drop(columns=['timeOpen', 'timeClose', 'timeHigh', 'timeLow'], inplace=True)
scaled_df = scaled_df.set_index('timestamp')

print(scaled_df.head().dtypes)

name           int64
open         float64
high         float64
low          float64
close        float64
volume       float64
marketCap    float64
dtype: object


In [3]:
def prepare_data(df, window_size=5):
    X, y = [], []
    for i in range(len(df) - window_size):
        features = df.iloc[i:i+window_size].drop(columns=['close']).values
        target = df.iloc[i+window_size]['close']
        X.append(features.flatten())
        y.append(target)
    return np.array(X), np.array(y)

  
# X_btc, y_btc = prepare_data(df_btc)
# X_eth, y_eth = prepare_data(df_eth)

# X_train_btc, X_test_btc, y_train_btc, y_test_btc = train_test_split(X_btc, y_btc, test_size=0.2, shuffle=False)
# X_train_eth, X_test_eth, y_train_eth, y_test_eth = train_test_split(X_eth, y_eth, test_size=0.2, shuffle=False)

# print(f"{X_train_btc.shape=}, {X_test_btc.shape=}, {y_train_btc.shape=}, {y_test_btc.shape=}")
# print(f"{X_train_eth.shape=}, {X_test_eth.shape=}, {y_train_eth.shape=}, {y_test_eth.shape=}")

# X_train, X_test, y_train, y_test = X_train_btc, X_test_btc, y_train_btc, y_test_btc

In [4]:

import torch.nn as nn

class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MLPModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, 1)
        )

    def forward(self, x):
        return self.model(x)


In [5]:

X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


NameError: name 'X_train' is not defined

In [ ]:
results = {}

for i in range(5, 40, 5):
    X, y = prepare_data(scaled_df, window_size=5)
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)

    
    input_size = i
    hidden_size = min(128, 2 * i)
    model = MLPModel(input_size=input_size, hidden_size=hidden_size)

    run_torch_model(f"MLP_{i}", model, X_train, y_train, X_test, y_test, scaler, results)

print("\n📈 RMSE Results Summary:")
for model_name, rmse in results.items():
    print(f"{model_name:20s}: RMSE = {rmse:.4f}")


Training MLP_5...


RuntimeError: mat1 and mat2 shapes cannot be multiplied (315x30 and 5x10)